In [4]:
import numpy as np

# Đường dẫn tới file .npz
file_path = 'data/NYC/bike_combined/test.npz'

# Đọc file .npz
data = np.load(file_path)

# In tên và hình dạng của từng mảng trong file
for key in data.files:
    print(f"{key}: shape = {data[key].shape}")


x: shape = (672, 12, 250, 4)
y: shape = (672, 12, 250, 2)
x_offsets: shape = (12, 1)
y_offsets: shape = (12, 1)
